### Imports

In [1]:
import sqlite3 as sql
import pandas as pd
import torch.nn as nn
import torch
import numpy as np
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sklearn.preprocessing import OneHotEncoder

conn = sql.connect("states.db")
tables_query = "SELECT name FROM sqlite_master WHERE type='table';"
tables_df = pd.read_sql(tables_query, conn)
table_names = tables_df['name'].tolist()[1:]


dataframes = {}
for table_name in table_names:
    dataframes[table_name] = pd.read_sql(f"SELECT * FROM {table_name}", conn)
conn.close()

In [14]:
unemployment_rate = dataframes['unemployment_rate']
state_survival_rates = dataframes['survival_rates']
industry_survival_rates = dataframes['industry_survival_rates']
establishments = dataframes['establishments']

### Data reformatting

Reformat into 1530 samples for state data

In [ ]:
state_survival_rates_filtered = state_survival_rates[state_survival_rates["Year Established"] != state_survival_rates["Year"]]
state_survival_rates_filtered.loc[:, "Year"] = state_survival_rates_filtered["Year"].astype('int')
state_survival_rates_filtered.loc[:, "Survival Rates of Previous Year's Survivors"] = state_survival_rates_filtered["Survival Rates of Previous Year's Survivors"].astype('float')
state_survival_rates_filtered.loc[:, 'Surviving Establishments'] = state_survival_rates_filtered['Surviving Establishments'].str.replace(',', '').astype(int)
state_survival_rates_grouped = state_survival_rates_filtered.groupby(["Year", "State"]).apply(lambda x: np.average(x['Survival Rates of Previous Year\'s Survivors'], weights=x['Surviving Establishments']))
#print(state_survival_rates_filtered.head())

Get reformatted industry data

In [28]:
industry_survival_rates_filtered = industry_survival_rates[industry_survival_rates["Year Established"] != industry_survival_rates["Year"]]
industry_survival_rates_filtered.loc[:, "Year"] = industry_survival_rates_filtered["Year"].astype('int')
industry_survival_rates_filtered.loc[:, "Survival Rates of Previous Year's Survivors"] = industry_survival_rates_filtered["Survival Rates of Previous Year's Survivors"].astype('float')
industry_survival_rates_filtered.loc[:, 'Surviving Establishments'] = industry_survival_rates_filtered['Surviving Establishments'].str.replace(',', '').astype(int)
industry_survival_rates_grouped = industry_survival_rates_filtered.groupby(["Year", "Industry"]).apply(lambda x: np.average(x['Survival Rates of Previous Year\'s Survivors'], weights=x['Surviving Establishments']))

print(industry_survival_rates_filtered.head())

  Year Established  Year Surviving Establishments  \
1             1994  1995                     5537   
2             1994  1996                     4826   
3             1994  1997                     4406   
4             1994  1998                     4005   
5             1994  1999                     3729   

  Total Employment of Survivors Survival Rates Since Birth  \
1                        51,066                       82.1   
2                        48,263                       71.6   
3                        47,634                       65.4   
4                        45,444                       59.4   
5                        39,377                       55.3   

  Survival Rates of Previous Year's Survivors Average Employment of Survivors  \
1                                        82.1                             9.2   
2                                        87.2                            10.0   
3                                        91.3                    

In [27]:
total_survival_rate = pd.concat([state_survival_rates_filtered, industry_survival_rates_filtered])
total_survival_rate_grouped = total_survival_rate.groupby(["Year"]).apply(lambda x: np.average(x['Survival Rates of Previous Year\'s Survivors'], weights=x['Surviving Establishments']))
# ^ overall business survival rate by year



### Bayesian Approach

In [33]:
import pandas as pd
import numpy as np

def compute_bayesian_survival_probabilities(state_survival_rates_filtered, industry_survival_rates_filtered):
    probabilities = []
    years = state_survival_rates_filtered['Year'].unique() # loop through the unique years in the list

    # P(Survival | State, Sector) = P(Survival) * P(State, Sector | Survival)P(State, Sector)
    
    for year in years:
        df_s = state_survival_rates_filtered[state_survival_rates_filtered['Year'] == year] # data during the current iterated year
        df_i = industry_survival_rates_filtered[industry_survival_rates_filtered['Year'] == year]
        df_state = df_s.copy()
        df_industry = df_i.copy()
       
        df_state.loc[:, 'Total Establishments'] = df_s['Surviving Establishments'].shift(1).fillna(df_s['Surviving Establishments'])
        # total establishments is just the surviving number from the last year
        df_industry.loc[:, 'Total Establishments'] = df_i['Surviving Establishments'].shift(1).fillna(df_i['Surviving Establishments'])
        # making sure the first one has data to shift down also
        
        total_state_survivors = df_state['Surviving Establishments'].sum()
        total_industry_survivors = df_industry['Surviving Establishments'].sum()
        total_survivors = (total_state_survivors + total_industry_survivors) / 2  # still using weighted average for now
        
        total_state_establishments = df_state['Total Establishments'].sum()
        total_industry_establishments = df_industry['Total Establishments'].sum()
        total_establishments = (total_state_establishments + total_industry_establishments) / 2
        
        p_survival = total_survivors / total_establishments if total_establishments > 0 else 0  # P(Survival)
        
        # Compute P(State, Sector) - probability of being in a state-sector
        df_state.loc[:, 'P_State'] = df_state['Total Establishments'] / total_state_establishments
        df_industry.loc[:, 'P_Sector'] = df_industry['Total Establishments'] / total_industry_establishments
        
        # Compute P(State | Survival) and P(Sector | Survival)
        df_state['P_State_given_Survival'] = df_state['Surviving Establishments'] / total_state_survivors
        df_industry['P_Sector_given_Survival'] = df_industry['Surviving Establishments'] / total_industry_survivors
        
        # Merge datasets on state and industry
        df_merged = pd.merge(df_state, df_industry, on='Year', suffixes=('_State', '_Sector'))
        
        # Compute P(Survival | State, Sector) using Bayesian formula
        df_merged['P_Survival_given_State_Sector'] = (
            p_survival * df_merged['P_State_given_Survival'] * df_merged['P_Sector_given_Survival'] /
            (df_merged['P_State'] * df_merged['P_Sector'])
        ).fillna(0)  # Handle divide-by-zero cases
        
        probabilities.append(df_merged[['Year', 'State', 'Industry', 'P_Survival_given_State_Sector']])
    
    return pd.concat(probabilities)

# Example usage

bayesian_survival_probabilities = compute_bayesian_survival_probabilities(state_survival_rates_filtered, industry_survival_rates_filtered)
print(bayesian_survival_probabilities.head())


   Year State                                       Industry  \
0  1995    AL     Agriculture, Forestry, Fishing and Hunting   
1  1995    AL  Mining, Quarrying, and Oil and Gas Extraction   
2  1995    AL                                      Utilities   
3  1995    AL                                   Construction   
4  1995    AL                                  Manufacturing   

   P_Survival_given_State_Sector  
0                       0.976189  
1                       0.254581  
2                       0.319087  
3                     100.164821  
4                       0.442390  


### Non-Bayesian approach

In [44]:
results = []
for (year, state), state_val in state_survival_rates_grouped.items():
    for (ind_year, industry), ind_val in industry_survival_rates_grouped.items():
        if year == ind_year:
            result = (year, state, industry), (state_val * ind_val) / total_survival_rate_grouped[year]
            results.append(result)

result = pd.Series(dict(results))
unemployment_rate["Year"] = unemployment_rate["Year"].astype('int')

In [46]:
state_df = state_survival_rates_grouped.unstack()
industry_df = industry_survival_rates_grouped.unstack()

In [47]:
numerical_state = dict(zip(state_df.columns, range(len(state_df.columns))))
numerical_state_rev = dict(zip(range(len(state_df.columns)), state_df.columns))
numerical_industry = dict(zip(industry_df.columns, range(len(industry_df.columns))))
numerical_industry_rev = dict(zip(range(len(industry_df.columns)), industry_df.columns))

In [51]:
pre_df = []
for year in result.index.get_level_values(0).unique():
    for state in state_df.columns:
        unemployment = unemployment_rate[(unemployment_rate["Year"] == year) & (unemployment_rate["State"] == state)]["Unemployment Rate"]
        for industry in industry_df.columns:
            response = result[year, state, industry]
            pre_df.append((year, numerical_state[state], numerical_industry[industry], float(unemployment.iloc[0]), response))
final_dataset = pd.DataFrame(pre_df, columns=["Year", "State", "Industry", "Unemployment Rate", "Response"])
             

### Model

In [54]:
class EconDataset(Dataset):
    def __init__(self, data, state_col, industry_col, unemployment_col, response_col):
        self.data = data.copy()
        self.state_col = state_col
        self.industry_col = industry_col
        self.unemployment_col = unemployment_col
        self.response_col = response_col
        self.state_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
        self.industry_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
        self._preprocess_data()

    def _preprocess_data(self):
        self.encoded_states = self.state_encoder.fit_transform(self.data[[self.state_col]]) # Applying one-hot encoding to the state column
        self.encoded_industries = self.industry_encoder.fit_transform(self.data[[self.industry_col]]) # Applying one-hot encoding to the industry
        self.unemployment_stats = self.data[self.unemployment_col].values.reshape(-1, 1) # Turns into column vector
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        state = self.encoded_states[idx]
        industry = self.encoded_industries[idx]
        unemployment = self.unemployment_stats[idx]
        predictor = np.concatenate((unemployment, state, industry), axis=0)
        response = self.data[self.response_col].values[idx]
        return torch.tensor(predictor, dtype=torch.float32), torch.tensor(response, dtype=torch.float32).view(1) #Response is reshaped to a column vector.

In [60]:
SurvivalData = EconDataset(final_dataset, "State", "Industry", "Unemployment Rate", "Response")
dataloader = DataLoader(SurvivalData, batch_size=2, shuffle=True)  # shuffle = True is important for training


In [62]:
class SurvivalRateModel(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size = 1):
        super(SurvivalRateModel, self).__init__()

        # Define layers:
        self.layers = nn.Sequential(
            nn.Linear(input_size, hidden_size1),
            nn.ReLU(),
            nn.Linear(hidden_size1, hidden_size2),
            nn.ReLU(),
            nn.Linear(hidden_size2, output_size)
        )

    def forward(self, x):
        return self.layers(x)

    
    def train_step(self, x, y, criterion, optimizer):
        optimizer.zero_grad()
        outputs = self(x)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()
        return loss.item()
    
    def train_loop(self, dataloader, num_epochs=10, learning_rate=0.001, device = "cpu"):
        self.to(device)
        criterion = nn.MSELoss()
        optimizer = torch.optim.Adam(self.parameters(), lr=learning_rate)
        self.train() #set model to train mode.
        for epoch in range(num_epochs):
            total_loss = 0
            for x, y in dataloader:
                x, y = x.to(device), y.to(device)
                loss = self.train_step(x, y, criterion, optimizer)
                total_loss += loss
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(dataloader):.4f}')
    

In [64]:
data = EconDataset(final_dataset, "State", "Industry", "Unemployment Rate", "Response")
dataloader = DataLoader(data, batch_size=2, shuffle=True)
Model = SurvivalRateModel(input_size=1 + len(data.state_encoder.categories_[0]) + len(data.industry_encoder.categories_[0]), hidden_size1=4, hidden_size2=8)

In [66]:
Model.train_loop(dataloader, num_epochs=10, learning_rate=0.001, device="cpu")

Epoch [1/10], Loss: 393.2532
Epoch [2/10], Loss: 13.7737
Epoch [3/10], Loss: 13.7004
Epoch [4/10], Loss: 13.6063
Epoch [5/10], Loss: 13.4531
Epoch [6/10], Loss: 13.3315
Epoch [7/10], Loss: 13.1304
Epoch [8/10], Loss: 13.0063
Epoch [9/10], Loss: 12.7960
Epoch [10/10], Loss: 12.6916
